In [1]:
import os
import sys
home = os.path.abspath('..')
os.chdir(home)
sys.path.append(home)

import pandas as pd_real
import modin.pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
def get_bounds(x):
    matches = re.findall("\d+",x)
    matches = list(map(float,matches))
        
    if len(matches)==0:
        matches = [float("-inf"),float("inf")]
    
    if len(matches)==1:
        matches = matches+[float("inf")]
    
    return matches

In [51]:
df = pd.read_csv('data/processed/processed_2019.csv')

In [52]:
rare = set(df["EMPLOYER_STATE"].dropna()) - set(states['Code'])
a = df["EMPLOYER_STATE"].value_counts(normalize = True)*100

In [50]:
a[a.index.isin(rare)]

PR    0.104045
MP    0.058897
GU    0.055498
VI    0.009910
FM    0.000181
AS    0.000181
dtype: float64

In [16]:
sum(df['WAGE_RATE_OF_PAY_FROM']<80000)*100/len(df)

32.119612840219666

In [19]:
df['FULL_TIME_POSITION.1'].value_counts()

Y    587741
N     10255
dtype: int64

In [20]:
states = pd.read_csv("data/external/states.csv")


In [212]:
person_income = pd.read_csv("data/external/personal_income.csv",)

In [213]:
person_income['group'] = person_income['group'].str.replace(',','').str.replace('$','')
person_income['frequency'] = person_income['frequency'].str.replace(',','').str.replace('$','').astype(int)

In [214]:
person_income[["lower_bound",
               "upper_bound"]] = pd.DataFrame(person_income['group']\
                                                            .apply(get_bounds)\
                                                            .to_list())

In [220]:
person_income[1:].to_csv("data/processed/personal_income.csv",index=False)

In [8]:
a = [i for i in df['EMPLOYER_NAME'].str.lower() if 'microsoft' in str(i)]

In [9]:
df.loc[df['EMPLOYER_NAME'].str.lower().isin(a), 'EMPLOYER_NAME'].value_counts()

MICROSOFT CORPORATION                    6040
MICROSOFT OPERATIONS PUERTO RICO, LLC       1
MICROSOFT                                   1
dtype: int64

In [11]:
df[df['EMPLOYER_NAME'] == 'MICROSOFT CORPORATION']

,Unnamed: 0,SOC_CODE,CASE_NUMBER,CASE_STATUS,FULL_TIME_POSITION,PERIOD_OF_EMPLOYMENT_START_DATE,PERIOD_OF_EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_STATE,WAGE_RATE_OF_PAY_FROM_1,WAGE_UNIT_OF_PAY_1,NAICS_CODE,FULL_TIME_POSITION.1
354,633,11.0,I-200-18268-812197,CERTIFIED,Y,3/16/19,3/15/22,MICROSOFT CORPORATION,WA,181813.0,Year,51121.0,Y
774,1078,11.0,I-200-19009-988705,CERTIFIED,Y,6/25/19,6/24/22,MICROSOFT CORPORATION,WA,186909.0,Year,51121.0,Y
3236,3673,11.0,I-200-19059-433559,CERTIFIED,Y,3/18/19,3/17/22,MICROSOFT CORPORATION,WA,174420.0,Year,51121.0,Y
12625,14078,11.0,I-200-18274-855269,CERTIFIED,Y,10/22/2018,10/21/2021,MICROSOFT CORPORATION,WA,123000.0,Year,51121.0,Y
12771,14230,11.0,I-200-18319-210629,CERTIFIED,Y,05/17/2019,05/16/2022,MICROSOFT CORPORATION,WA,160600.0,Year,51121.0,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
596620,663149,41.0,I-200-19150-546610,CERTIFIED,Y,7/17/19,7/16/22,MICROSOFT CORPORATION,WA,133578.0,Year,51121.0,Y
596671,663201,41.0,I-200-19157-262687,CERTIFIED,Y,6/13/19,6/12/22,MICROSOFT CORPORATION,WA,104000.0,Year,51121.0,Y
596882,663417,41.0,I-200-19196-882956,CERTIFIED,Y,7/26/19,7/26/22,MICROSOFT CORPORATION,WA,140104.0,Year,51121.0,Y
596892,663427,41.0,I-200-19197-648761,CERTIFIED,Y,1/15/20,1/14/23,MICROSOFT CORPORATION,WA,122593.0,Year,51121.0,Y


In [32]:
%%time
df = df.dropna(how = 'all',axis=0).dropna(how = 'all',axis=1)

CPU times: user 2.8 s, sys: 1.08 s, total: 3.88 s
Wall time: 1min 53s


In [48]:
na_cols = (df.isna().sum(axis=0)*100/len(df)).sort_values(ascending=False)

In [50]:
p = 99
print(f"{sum(na_cols>p)} columns are missing {p}% of the values")

47 columns are missing 50% of the values


In [44]:
#Remove empty columns
empty_cols = list(na_cols.index[na_cols>p])
df.drop(empty_cols,axis=1,inplace=True)

In [54]:
df.to_csv('data/interim/H1B_FY2019.csv')

In [2]:
df = pd.read_csv('data/interim/H1B_FY2019.csv')

In [3]:
#Very rare worksite locations info
cols = [re.match('.*_[2-9]',i).group() for i in df.columns if re.match('.*_[2-9]',i) is not None]

In [4]:
df.drop(cols,axis=1,inplace=True)

In [5]:
status_mask = (df['CASE_STATUS'] == 'CERTIFIED') | (df['CASE_STATUS'] == 'WITHDRAWN')

In [6]:
df = df[status_mask]

In [7]:
df['CASE_STATUS'].value_counts()*100/len(df)

CERTIFIED    96.784122
WITHDRAWN     3.215878
dtype: float64

In [8]:
df['CASE_STATUS'].value_counts()

CERTIFIED    592103
WITHDRAWN     19674
dtype: int64

In [131]:
df['SOC_TITLE'].value_counts()

SOFTWARE DEVELOPERS, APPLICATIONS        200519
COMPUTER OCCUPATIONS, ALL OTHER           52865
COMPUTER SYSTEMS ANALYST                  40836
COMPUTER SYSTEMS ANALYSTS                 29301
SOFTWARE DEVELOPERS, SYSTEMS SOFTWARE     29150
                                          ...  
ACCOUNTS AND AUDITORS                         1
ACCOUNTANTS AND AUDOTORS                      1
17-2072                                       1
15-2031                                       1
15-1121                                       1
Length: 886, dtype: int64

In [10]:
df[['NEW_EMPLOYMENT','CONTINUED_EMPLOYMENT','CHANGE_PREVIOUS_EMPLOYMENT','NEW_CONCURRENT_EMPLOYMENT']].sum()

NEW_EMPLOYMENT                303034.0
CONTINUED_EMPLOYMENT          141510.0
CHANGE_PREVIOUS_EMPLOYMENT     74299.0
NEW_CONCURRENT_EMPLOYMENT       8518.0
dtype: float64

In [22]:
df['PW_WAGE_LEVEL_1'].value_counts(normalize = True)

Level II     0.530597
Level III    0.196667
Level I      0.156490
Level IV     0.116247
dtype: float64

In [23]:
status_mask = df['CASE_STATUS'] == 'CERTIFIED'
df.loc[status_mask,'PW_WAGE_LEVEL_1'].value_counts(normalize = True)

Level II     0.530485
Level III    0.197595
Level I      0.155482
Level IV     0.116438
dtype: float64

In [24]:
df.loc[~status_mask,'PW_WAGE_LEVEL_1'].value_counts(normalize = True)

Level II     0.503389
Level I      0.196362
Level III    0.180115
Level IV     0.120134
dtype: float64

In [27]:
df['WILLFUL_VIOLATOR'].value_counts()

N    598012
Y       250
dtype: int64

In [32]:
df.loc[status_mask,'WILLFUL_VIOLATOR'].value_counts(normalize = True)

N    0.999661
Y    0.000339
dtype: float64

In [33]:
df.loc[~status_mask,'WILLFUL_VIOLATOR'].value_counts(normalize = True)

N    0.997327
Y    0.002673
dtype: float64

In [97]:
for col in ['PW_WAGE_LEVEL_1']:
    
    print(col)
    contingency = pd.crosstab(df['CASE_STATUS'],
                        df[col],
                        margins = False)
    p_val = chi2_contingency(contingency)[1]
    print(col,' ',p_val)

PW_WAGE_LEVEL_1


PW_WAGE_LEVEL_1   4.839464645531454e-46


In [106]:
df['WORKSITE_STATE_1'].value_counts()

CALIFORNIA          110588
TEXAS                56078
NEW YORK             47848
NEW JERSEY           33024
ILLINOIS             27641
                     ...  
MP                       6
VI                       4
PW                       1
PALAU                    1
MARSHALL ISLANDS         1
Length: 113, dtype: int64

In [108]:
df['EMPLOYER_STATE'].value_counts()

CA    110229
TX     94217
NJ     75872
NY     40995
IL     36602
PA     25273
WA     23959
MA     23799
MI     21409
MD     18895
VA     17547
NC     15628
GA     14726
FL     13915
OH      8566
CT      5867
TN      5540
MO      5077
MN      4843
AZ      4342
CO      3705
AR      3445
IN      3334
WI      2994
DE      2984
UT      2256
SC      2197
IA      2140
KS      1966
DC      1957
NE      1890
KY      1754
OR      1698
AL      1453
OK      1317
LA      1215
NV      1209
RI      1102
NH      1088
ID       755
NM       698
MS       462
ME       437
WV       373
VT       337
GU       331
ND       320
HI       268
SD       203
WY       155
AK       110
MP        93
MT        83
PR        74
VI        38
PW         1
FM         1
dtype: int64

In [112]:
df['EMPLOYER_NAME'].value_counts()[0:20]

COGNIZANT TECHNOLOGY SOLUTIONS US CORP    28454
INFOSYS LIMITED                           21445
TATA CONSULTANCY SERVICES LIMITED         11861
GOOGLE LLC                                 9467
ERNST & YOUNG U.S. LLP                     8739
CAPGEMINI AMERICA INC                      7822
DELOITTE CONSULTING LLP                    7745
AMAZON.COM SERVICES, INC.                  7206
MICROSOFT CORPORATION                      6040
ACCENTURE LLP                              5555
IBM CORPORATION                            4906
HCL AMERICA, INC.                          4621
WIPRO LIMITED                              4242
LARSEN & TOUBRO INFOTECH LIMITED           3324
TECH MAHINDRA (AMERICAS),INC.              2915
FACEBOOK, INC.                             2316
WAL-MART ASSOCIATES, INC.                  2116
IBM INDIA PRIVATE LIMITED                  1991
L&T TECHNOLOGY SERVICES LIMITED            1862
SYNTEL INC                                 1731
dtype: int64

In [113]:
len(df['EMPLOYER_NAME'].unique())

67454

In [117]:
len(df['NAICS_CODE'].unique())

1990

In [118]:
df['NAICS_CODE'].value_counts()

541511.0    195013
541512.0     62802
611310.0     21974
5416.0       18684
54151.0      15860
             ...  
3241.0           1
3122.0           1
3112.0           1
1122.0           1
31.0             1
Length: 1989, dtype: int64

In [22]:
df['NAICS_CODE'].astype(str).str.replace('.0','').apply(len).value_counts()

6    360774
4    185551
5     51658
3         9
2         4
dtype: int64

In [23]:
df['NAICS_CODE'][0:5]

0    511210.0
1    541810.0
2     56142.0
3    541810.0
4    541810.0
Name: NAICS_CODE, dtype: float64

In [123]:
df.loc[df['NAICS_CODE']==541511.0,'EMPLOYER_NAME'].value_counts()

INFOSYS LIMITED                      21445
TATA CONSULTANCY SERVICES LIMITED    11861
HCL AMERICA, INC.                     4620
WIPRO LIMITED                         4242
TECH MAHINDRA (AMERICAS),INC.         2915
                                     ...  
24/7 SOFTWARE, INC.                      1
1UPHEALTH, INC.                          1
1800ADMIN, LLC.                          1
12 SIGMA TECHNOLOGIES                    1
1 WAY SOLUTIONS, INC.                    1
Length: 9585, dtype: int64

In [124]:
df.loc[df['NAICS_CODE']==541512.0,'EMPLOYER_NAME'].value_counts()

COGNIZANT TECHNOLOGY SOLUTIONS US CORP    28454
GOOGLE LLC                                 9436
POLARIS CONSULTING & SERVICES LTD           617
HEXAWARE TECHNOLOGIES, INC.                 587
LARSEN & TOUBRO INFOTECH LIMITED            539
                                          ...  
A.J. BOGGS &AMP; COMPANY                      1
802 SECURE INC.                               1
42 TECHNOLOGIES, INC.                         1
3TECH CORPORATION                             1
1                                             1
Length: 2143, dtype: int64

In [126]:
df.loc[df['NAICS_CODE']==611310.0,'EMPLOYER_NAME'].value_counts()[0:20]

UNIVERSITY OF MICHIGAN                                          376
MAYO CLINIC                                                     366
THE BOARD OF TRUSTEES OF THE LELAND STANFORD, JR. UNIVERSITY    339
COLUMBIA UNIVERSITY                                             258
UNIVERSITY OF CALIFORNIA, SAN FRANCISCO                         255
TRUSTEES OF THE UNIVERSITY OF PENNSYLVANIA                      255
UNIVERSITY OF PITTSBURGH                                        250
WASHINGTON UNIVERSITY IN ST. LOUIS                              231
UNIVERSITY OF FLORIDA                                           231
UNIVERSITY OF MINNESOTA                                         228
HARVARD UNIVERSITY                                              211
UNIVERSITY OF CALIFORNIA, LOS ANGELES                           210
THE UNIVERSITY OF SOUTHERN CALIFORNIA                           209
THE OHIO STATE UNIVERSITY                                       207
YALE UNIVERSITY                                 

In [158]:
unis = [i for i in df['EMPLOYER_NAME'] if 'UNIVERSITY' in str(i)]

In [162]:
df.loc[df['EMPLOYER_NAME'].isin(unis),'NAICS_CODE'].value_counts()

611310.0    18871
622110.0      513
61131.0       259
6113.0        240
621111.0      125
62111.0        96
541710.0       60
6221.0         41
6211.0         26
923110.0       16
5417.0         15
92311.0        12
54171.0        11
611710.0       10
541712.0        9
813211.0        8
624190.0        8
611210.0        8
611110.0        8
541990.0        8
523920.0        7
541810.0        3
522130.0        3
813110.0        2
611610.0        2
518210.0        2
813410.0        1
622210.0        1
621493.0        1
621399.0        1
621210.0        1
621112.0        1
541618.0        1
541513.0        1
541511.0        1
238990.0        1
115112.0        1
81321.0         1
62211.0         1
56111.0         1
54138.0         1
54133.0         1
44531.0         1
6215.0          1
6117.0          1
5259.0          1
dtype: int64

In [13]:
df.columns

Index(['Unnamed: 0', 'CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED',
       'DECISION_DATE', 'ORIGINAL_CERT_DATE', 'VISA_CLASS', 'JOB_TITLE',
       'SOC_CODE', 'SOC_TITLE',
       ...
       'WAGE_UNIT_OF_PAY_3', 'PREVAILING_WAGE_3', 'PW_UNIT_OF_PAY_3',
       'PW_WAGE_LEVEL_3', 'PW_NON-OES_YEAR_3', 'H-1B_DEPENDENT',
       'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'STATUTORY_BASIS',
       'PUBLIC_DISCLOSURE'],
      dtype='object', length=101)

In [151]:
df.loc[df['EMPLOYER_NAME'].isin(['UNIVERSITY']),'NAICS_CODE'].value_counts()

Series([], Name: NAICS_CODE, dtype: int64)